In [2]:
import torch
import torch.nn as nn
import numpy as np

# Softmax

In [4]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy:', outputs)

softmax numpy: [0.65900114 0.24243297 0.09856589]


In [5]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print('softmax torch: ', outputs)

softmax torch:  tensor([0.6590, 0.2424, 0.0986])


# Cross-Entropy Loss

In [7]:
# numpy cross entropy
def cross_entropy(actual, predict):
    loss = -np.sum(actual * np.log(predict))
    return loss

Y = np.array([1, 0, 0]) # one-hot encoded labels

# Y_pred: probabilities
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss1 numpy: {l1}')
print(f'Loss2 numpy: {l2}')

Loss1 numpy: 0.35667494393873245
Loss2 numpy: 2.3025850929940455


In [9]:
# pytorch cross entropy
# nn.CrossEntropyLoss : nn.LogSoftmax + nn.NLLLoss (negative log likelihood loss)
# 마지막에 softmax 넣을 필요 없음
# Y -> class labels (not one-hot), 정답 class label값만 넣어줌. one-hot으로 정답 class label만 1로 할 필요 없음
# Y_pred -> raw scores(logits) not softmax probabilities
loss = nn.CrossEntropyLoss()

Y = torch.tensor([0])   # Correct last label, not one-hot
# n_samples * n_classes = 1 * 3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])   # raw scores
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss1 torch: {l1.item()}')
print(f'Loss2 torch: {l2.item()}')

_, predictions1 = torch.max(Y_pred_good, 1)     # Show highest score class
_, predictions2 = torch.max(Y_pred_bad, 1)
print(predictions1, predictions2)

Loss1 torch: 0.4170299470424652
Loss2 torch: 1.840616226196289
tensor([0]) tensor([1])


In [12]:
# 3 samples
Y = torch.tensor([2, 0, 1])
Y_pred_good = torch.tensor([[0.1, 1.0, 2.1], [2.0, 1.0, 0.1], [0.1, 3.0, 0.1]])
Y_pred_bad = torch.tensor([[2.0, 1.0, 0.1], [0.1, 1.0, 2.1], [0.1, 3.0, 0.1]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss1 torch: {l1.item()}')
print(f'Loss2 torch: {l2.item()}')

_, predictions1 = torch.max(Y_pred_good, 1)     # Show highest score class
_, predictions2 = torch.max(Y_pred_bad, 1)
print(predictions1, predictions2)

Loss1 torch: 0.3018244206905365
Loss2 torch: 1.601824402809143
tensor([2, 0, 1]) tensor([0, 2, 1])


# Simple Neural Net

In [ ]:
# Multiclass problem
class NeuralNetMulti(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetMulti, self).__init__()
        self.linear1 =  nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(x)
        # No Softmax here
        return out

model = NeuralNetMulti(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss()   # Apply Softmax here

In [ ]:
# Binary Classification Problem
# Linear layer 후 출력 1개만 나옴
# Softmax 없이 Sigmoid 적용
# nn.BCELoss(), Sigmoid 마지막에 추가해야함
# Multiclass problem
class NeuralNetBinary(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetBinary, self).__init__()
        self.linear1 =  nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(x)
        y_pred = torch.sigmoid(out) # Apply Sigmoid here
        return y_pred

model = NeuralNetBinary(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()   # No sigmoid here